In [7]:
#library import
import tensorflow as tf
from tensorflow import keras

import os
import re
from PIL import Image
import shutil
import random

import numpy as np
import matplotlib.pyplot as plt
import math

print(tf.__version__)

2.4.1


In [15]:
#sample data site : https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
#dataset download

import gdown
#url = 'https://www.robots.ox.ac.uk/~vgg/data/pets/data/'
#filename = 'images.tar.gz'
#gdown.download(url, filename, quiet=False)

url = 'https://drive.google.com/uc?id=1dIR9ANjUsV9dWa0pS9J0c2KUGMfpIRG0'
filename = 'oxford_pet.zip'
gdown.download(url, filename, quiet=False)



Downloading...
From: https://drive.google.com/uc?id=1dIR9ANjUsV9dWa0pS9J0c2KUGMfpIRG0
To: /content/oxford_pet.zip
811MB [00:07, 108MB/s]


'oxford_pet.zip'

In [16]:
!ls -l

total 791576
-rw-r--r-- 1 root root 810565619 Apr 29 07:14 oxford_pet.zip
drwxr-xr-x 1 root root      4096 Apr 21 13:39 sample_data


In [17]:
#압축풀기
!unzip -q oxford_pet.zip -d oxford_pet

In [23]:
#directory setting
cur_dir = os.getcwd()
data_dir = os.path.join(cur_dir, 'oxford_pet')
image_dir = os.path.join(data_dir, 'images')

print(data_dir, image_dir)

/content/oxford_pet /content/oxford_pet/images


In [28]:
#image file count
image_files = [filename for filename in os.listdir(image_dir) if os.path.splitext(filename)[-1] == '.jpg']

print(len(image_files))

7390


In [30]:
#image file을 읽어서 channel이 3이 아닌 image는 삭제함
for image_file in image_files:
  image_path = os.path.join(image_dir, image_file)
  image = Image.open(image_path)
  image_mode = image.mode
  if image_mode != 'RGB':
    print(image_file, image_mode)
    image = np.asarray(image)
    print(image.shape)
    os.remove(image_path)

Egyptian_Mau_129.jpg L
(325, 299)
Egyptian_Mau_145.jpg P
(188, 216)
Egyptian_Mau_139.jpg P
(250, 350)
Abyssinian_34.jpg P
(202, 250)
Egyptian_Mau_14.jpg RGBA
(800, 582, 4)
Egyptian_Mau_177.jpg P
(175, 246)
Egyptian_Mau_186.jpg RGBA
(275, 183, 4)
staffordshire_bull_terrier_2.jpg L
(282, 500)
Abyssinian_5.jpg RGBA
(150, 200, 4)
Egyptian_Mau_167.jpg P
(275, 183)
Egyptian_Mau_191.jpg P
(214, 300)
staffordshire_bull_terrier_22.jpg L
(500, 364)


In [31]:
image_files = [filename for filename in os.listdir(image_dir) if os.path.splitext(filename)[-1] == '.jpg']

print(len(image_files))

7378


In [32]:
image_files[:10]

['english_cocker_spaniel_4.jpg',
 'pug_163.jpg',
 'american_bulldog_152.jpg',
 'newfoundland_107.jpg',
 'leonberger_197.jpg',
 'pug_149.jpg',
 'Ragdoll_27.jpg',
 'leonberger_20.jpg',
 'pug_12.jpg',
 'beagle_59.jpg']

In [34]:
class_list = set()
for image_file in image_files:
  file_name = os.path.splitext(image_file)[0]
  class_name = re.sub('_\d+', '', file_name)
  class_list.add(class_name)
class_list = list(class_list)
print(len(class_list))

class_list.sort()
class_list

37


['Abyssinian',
 'Bengal',
 'Birman',
 'Bombay',
 'British_Shorthair',
 'Egyptian_Mau',
 'Maine_Coon',
 'Persian',
 'Ragdoll',
 'Russian_Blue',
 'Siamese',
 'Sphynx',
 'american_bulldog',
 'american_pit_bull_terrier',
 'basset_hound',
 'beagle',
 'boxer',
 'chihuahua',
 'english_cocker_spaniel',
 'english_setter',
 'german_shorthaired',
 'great_pyrenees',
 'havanese',
 'japanese_chin',
 'keeshond',
 'leonberger',
 'miniature_pinscher',
 'newfoundland',
 'pomeranian',
 'pug',
 'saint_bernard',
 'samoyed',
 'scottish_terrier',
 'shiba_inu',
 'staffordshire_bull_terrier',
 'wheaten_terrier',
 'yorkshire_terrier']

In [38]:
class_list[1]

class2idx = {cls:idx for idx, cls in enumerate(class_list)}
class2idx

{'Abyssinian': 0,
 'Bengal': 1,
 'Birman': 2,
 'Bombay': 3,
 'British_Shorthair': 4,
 'Egyptian_Mau': 5,
 'Maine_Coon': 6,
 'Persian': 7,
 'Ragdoll': 8,
 'Russian_Blue': 9,
 'Siamese': 10,
 'Sphynx': 11,
 'american_bulldog': 12,
 'american_pit_bull_terrier': 13,
 'basset_hound': 14,
 'beagle': 15,
 'boxer': 16,
 'chihuahua': 17,
 'english_cocker_spaniel': 18,
 'english_setter': 19,
 'german_shorthaired': 20,
 'great_pyrenees': 21,
 'havanese': 22,
 'japanese_chin': 23,
 'keeshond': 24,
 'leonberger': 25,
 'miniature_pinscher': 26,
 'newfoundland': 27,
 'pomeranian': 28,
 'pug': 29,
 'saint_bernard': 30,
 'samoyed': 31,
 'scottish_terrier': 32,
 'shiba_inu': 33,
 'staffordshire_bull_terrier': 34,
 'wheaten_terrier': 35,
 'yorkshire_terrier': 36}

In [39]:
class2idx['Bengal']

1

In [40]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

image_files.sort()

image_files[:10]

['Abyssinian_1.jpg',
 'Abyssinian_10.jpg',
 'Abyssinian_100.jpg',
 'Abyssinian_101.jpg',
 'Abyssinian_102.jpg',
 'Abyssinian_103.jpg',
 'Abyssinian_104.jpg',
 'Abyssinian_105.jpg',
 'Abyssinian_106.jpg',
 'Abyssinian_107.jpg']

In [41]:
#train data, validation data
cnt = 0
previous_class = ""

for image_file in image_files:
  file_name = os.path.splitext(image_file)[0]
  class_name = re.sub('_\d+', '', file_name)
  if class_name == previous_class:
    cnt += 1
  else:
    cnt = 1
  if cnt <= 160:
    cpath = train_dir
  else:
    cpath = val_dir
  
  image_path = os.path.join(image_dir, image_file)
  shutil.copy(image_path, cpath)
  previous_class = class_name

In [42]:
train_image = os.listdir(train_dir)
val_image = os.listdir(val_dir)

In [43]:
train_image[:10]

['english_cocker_spaniel_4.jpg',
 'pug_163.jpg',
 'american_bulldog_152.jpg',
 'newfoundland_107.jpg',
 'leonberger_197.jpg',
 'pug_149.jpg',
 'Ragdoll_27.jpg',
 'leonberger_20.jpg',
 'pug_12.jpg',
 'beagle_59.jpg']

In [44]:
val_image[:10]

['keeshond_88.jpg',
 'staffordshire_bull_terrier_93.jpg',
 'pomeranian_89.jpg',
 'english_setter_89.jpg',
 'Persian_60.jpg',
 'newfoundland_90.jpg',
 'British_Shorthair_94.jpg',
 'scottish_terrier_81.jpg',
 'german_shorthaired_72.jpg',
 'american_bulldog_63.jpg']

In [ ]:
IMG_SIZE = 224

In [45]:
#TFRcord 저장할 directory와 file 경로 설정
tfr_dir = os.path.join(data_dir, 'tfrecord')
os.makedirs(tfr_dir, exist_ok=True)

tfr_train_dir = os.path.join(tfr_dir, 'cls_train.tfr')
tfr_val_dir = os.path.join(tfr_dir, 'cls_val.tfr')

In [46]:
#TFRcord writer create
writer_train = tf.io.TFRecordWriter(tfr_train_dir)
writer_val = tf.io.TFRecordWriter(tfr_val_dir)

In [47]:
#https://www.tensorflow.org/tutorials/load_data/tfrecord?hl=ko
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:

from tensorflow import keras
keras.layers.Conv2D(10, kernel_size=(3,3), activation='relu')

#padding
#입력 과 특성맵의 차원을 동일하게하고, same padding

keras.layers.Conv2D(10, kernel_size=(3,3), activation='relu', padding='same')

#stride는 겹치면서 한다
#이동하는 크기
keras.layers.Conv2D(10, kernel_size=(3,3), activation='relu', padding='same', strides=1)

#pooling은 겹치지 않는다.
#평균/최대 풀링을 많이 사용함
keras.layers.MaxPooling2D(2, strides=2, padding='same')
